In [1]:
#|default_exp mask_s2

# Masks S2

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const

Source : https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

In [4]:
#|export

def get_s2_sr_cld_col(fc : ee.featurecollection , # FeatureCollection with geometry datata
                      start_date : str , # Start date for searching images. , 
                      end_date : str , # End date for searching images.
                      ):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection(const.s2_img_collection_str)
        .filterBounds(fc)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte(const.CLOUDY_PIXEL_PERCENTAGE_STR, const.CLOUD_FILTER)))


    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection(const.COPERNICUS_S2_CLOUD_PROBABILITY_STR)
        .filterBounds(fc)
        .filterDate(start_date, end_date))
    
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst(const.s2cloudless_str).apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))




In [ ]:
#|export
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get(const.s2cloudless_str)).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(const.CLD_PRB_THRESH).rename(const.clouds_name_bands_str)

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
#|export

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select(const.SCL_str).neq(const.SCL_VALUE_int)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = const.SR_BAND_SCALE_int
    dark_pixels = img.select(const.B8_str).lt(const.NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename(const.dark_pixels_str)

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(const.direction_int).subtract(ee.Number(img.get(const.MEAN_SOLAR_AZIMUTH_ANGLE_str)))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select(const.clouds_name_bands_str).directionalDistanceTransform(shadow_azimuth, const.CLD_PRJ_DIST* const.int_10)
        .reproject(**{'crs': img.select(0).projection(), const.scale_str: const.scale_str_size})
        .select(const.distance_str)
        .mask()
        .rename(const.cloud_transform_str))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename(const.shadows_str)

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
#|export

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select(const.clouds_name_bands_str).add(img_cloud_shadow.select(const.shadows_str)).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(const.BUFFER*2/20)
        .reproject(**{const.crs_str: img.select([0]).projection(), const.scale_str: const.scale_2_int})
        .rename(const.cloudmask_str))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)